In [1]:
# pip install -q datasets


In [2]:
# pip install -q transformers

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.figure_factory as ff
from transformers import TFAutoModel
from transformers import TrainingArguments


In [4]:
tTrain = pd.read_csv("../data/NLP_Train.csv")
tTrain.head()

,id,text,keywords_label,fixed_text
0,579785953885040641,Cross eyed and tired. Long day. #LymeDisease h...,1,cross eyed and tired long day lymedisease
1,597900765995241472,@skyemyhusky Please help prevent lyme disease ...,1,please help prevent lyme disease in dogs thru ...
2,260534112660631552,@lymethriving TOUCHED BY LYME: Here’s video fr...,1,touched by lyme heres video from san diegos ly...
3,1129169681326256133,A new case report describes a 46 year old with...,1,a new case report describes a 46 year old with...
4,802284796420456448,#FunFactFriday Lyme disease is named for the c...,1,funfactfriday lyme disease is named for the ci...


In [5]:
tTrain = tTrain[['fixed_text', 'keywords_label']]
tTrain.head()

,fixed_text,keywords_label
0,cross eyed and tired long day lymedisease,1
1,please help prevent lyme disease in dogs thru ...,1
2,touched by lyme heres video from san diegos ly...,1
3,a new case report describes a 46 year old with...,1
4,funfactfriday lyme disease is named for the ci...,1


In [6]:
# bertweet = TFAutoModel.from_pretrained("vinai/bertweet-base")

In [8]:
# bertweet.summary()

In [9]:
# https://medium.com/mlearning-ai/fine-tuning-bert-for-tweets-classification-ft-hugging-face-8afebadd5dbf
from datasets import load_dataset
dataset = load_dataset('csv', data_files={'train': '../data/NLP_train.csv', 'test': '../data/NLP_test.csv'}, encoding = "ISO-8859-1")


Using custom data configuration default-76ef4e8b465acfe1
Reusing dataset csv (/Users/sxb592/.cache/huggingface/datasets/csv/default-76ef4e8b465acfe1/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'keywords_label', 'fixed_text'],
        num_rows: 70000
    })
    test: Dataset({
        features: ['id', 'text', 'keywords_label', 'fixed_text'],
        num_rows: 7500
    })
})

In [11]:
# tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False)

# # For transformers v3.x:
# # tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base")

# # INPUT TWEET IS ALREADY NORMALIZED!
# line = "SC has first two presumptive cases of coronavirus , DHEC confirms HTTPURL via @USER :cry:"

# input_ids = torch.tensor([tokenizer.encode(line)])

# with torch.no_grad():
#     features = bertweet(input_ids)  # Models outputs are now tuples

In [12]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

In [13]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [14]:
def transform_labels(label):

    num = label['keywords_label']
    return {'labels': num}

def tokenize_data(example):
    return tokenizer(example['fixed_text'], padding='max_length')

dataset = dataset.map(tokenize_data, batched=True)

remove_columns = ['id','text','keywords_label','fixed_text']
dataset = dataset.map(transform_labels, remove_columns=remove_columns)

Loading cached processed dataset at /Users/sxb592/.cache/huggingface/datasets/csv/default-76ef4e8b465acfe1/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-ed1e2e55db67d3d1.arrow


  0%|          | 0/8 [00:00<?, ?ba/s]

Loading cached processed dataset at /Users/sxb592/.cache/huggingface/datasets/csv/default-76ef4e8b465acfe1/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-9f402143ae1d9d24.arrow


  0%|          | 0/7500 [00:00<?, ?ex/s]

In [15]:
train_dataset = dataset['train'].shuffle(seed=10).select(range(3000))
eval_dataset = dataset['train'].shuffle(seed=10).select(range(3000, 3500))

Loading cached shuffled indices for dataset at /Users/sxb592/.cache/huggingface/datasets/csv/default-76ef4e8b465acfe1/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-f5a3b15ed9e5d741.arrow
Loading cached shuffled indices for dataset at /Users/sxb592/.cache/huggingface/datasets/csv/default-76ef4e8b465acfe1/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-f5a3b15ed9e5d741.arrow


In [18]:
from transformers import Trainer
from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer", num_train_epochs=1)


trainer = Trainer(
    model=model, args=training_args, train_dataset=train_dataset, eval_dataset=eval_dataset
)
trainer.train()


Step,Training Loss


TrainOutput(global_step=375, training_loss=0.3003210042317708, metrics={'train_runtime': 44454.7205, 'train_samples_per_second': 0.008, 'total_flos': 998222902272000.0, 'epoch': 1.0, 'init_mem_cpu_alloc_delta': 284976, 'init_mem_cpu_peaked_delta': 11636, 'train_mem_cpu_alloc_delta': 431884, 'train_mem_cpu_peaked_delta': 547906})

In [19]:
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer.evaluate()

{'eval_loss': 0.19101035594940186,
 'eval_runtime': 606.6956,
 'eval_samples_per_second': 0.824,
 'epoch': 1.0,
 'eval_mem_cpu_alloc_delta': 35101,
 'eval_mem_cpu_peaked_delta': 247575}

# BerTweet

In [22]:

from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import Trainer
from transformers import TrainingArguments


In [21]:
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-large")


Downloading:   0%|          | 0.00/614 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [1]:

# bertweet = AutoModel.from_pretrained("vinai/bertweet-large")
# tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-large")

model = AutoModelForSequenceClassification.from_pretrained("vinai/bertweet-large", num_labels=5)

train_dataset = dataset['train'].shuffle(seed=10).select(range(3000))
eval_dataset = dataset['train'].shuffle(seed=10).select(range(3000, 3500))



training_args = TrainingArguments("test_trainer", num_train_epochs=1)




NameError: name 'AutoModelForSequenceClassification' is not defined

In [ ]:

trainer = Trainer(
    model=model, args=training_args, train_dataset=train_dataset, eval_dataset=eval_dataset
)
trainer.train()


from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer.evaluate()